# MVD 9. cvičení

Dnešní cvičení nebude až tak obtížné. Cílem je seznámit se s HuggingFace a vyzkoušet si základní práci s BERT modelem.

## 1. část - Seznámení s HuggingFace a modelem BERT

Nainstalujte si Python knihovnu `transformers` a podívejte se na předtrénovaný [BERT model](https://huggingface.co/bert-base-uncased). Vyzkoušejte si unmasker s různými vstupy.

<br>
Pozn.: Použití BERT modelu vyžaduje zároveň PyTorch - postačí i cpu verze.

In [ ]:
import sys
!{sys.executable} -m pip install torch
!{sys.executable} -m pip install transformers

In [68]:
import torch
from transformers import pipeline
import plotly.express as px
from sklearn.manifold import TSNE

In [15]:
model_name = 'bert-base-uncased'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [16]:
unmasker = pipeline('fill-mask', model=model_name)
unmasker.model.to(device)

# vytiskne paramtery modelu
print(unmasker.model)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [27]:
def print_unmasker_output(output: list) -> None:
    for i, o in enumerate(output):
        print(f'{i+1}. {o["sequence"]} \t  {o["token_str"]}, ({o["score"]:.2f})')

In [35]:
filed = unmasker("Hello I'm a [MASK] model.", top_k=3)
print_unmasker_output(filed)

1. hello i'm a fashion model. 	  fashion, (0.11)
2. hello i'm a role model. 	  role, (0.09)
3. hello i'm a new model. 	  new, (0.05)


In [36]:
filed = unmasker("""
# Coding with BERT model
import [MASK] as np
""", top_k=10)
print_unmasker_output(filed)

1. # coding with bert model import code as np 	  code, (0.12)
2. # coding with bert model importer as np 	  ##er, (0.07)
3. # coding with bert model importing as np 	  ##ing, (0.06)
4. # coding with bert model imported as np 	  ##ed, (0.04)
5. # coding with bert model importable as np 	  ##able, (0.04)
6. # coding with bert model import codes as np 	  codes, (0.03)
7. # coding with bert model importation as np 	  ##ation, (0.02)
8. # coding with bert model import function as np 	  function, (0.01)
9. # coding with bert model import listed as np 	  listed, (0.01)
10. # coding with bert model import such as np 	  such, (0.01)


In [37]:
filed = unmasker("Romeo and Juliet is a [MASK] play.", top_k=3)
print_unmasker_output(filed)

1. romeo and juliet is a shakespeare play. 	  shakespeare, (0.22)
2. romeo and juliet is a musical play. 	  musical, (0.07)
3. romeo and juliet is a stage play. 	  stage, (0.04)


In [38]:
filed = unmasker("Romeo and Juliet is a [MASK] play.", top_k=10)
print_unmasker_output(filed)

1. romeo and juliet is a shakespeare play. 	  shakespeare, (0.22)
2. romeo and juliet is a musical play. 	  musical, (0.07)
3. romeo and juliet is a stage play. 	  stage, (0.04)
4. romeo and juliet is a theatrical play. 	  theatrical, (0.04)
5. romeo and juliet is a theatre play. 	  theatre, (0.03)
6. romeo and juliet is a french play. 	  french, (0.02)
7. romeo and juliet is a tragedy play. 	  tragedy, (0.02)
8. romeo and juliet is a contemporary play. 	  contemporary, (0.02)
9. romeo and juliet is a theater play. 	  theater, (0.01)
10. romeo and juliet is a modern play. 	  modern, (0.01)


In [42]:
unmasker("Continent of America was discovered in [MASK] by [MASK].", top_k=15)

[[{'score': 0.013635069131851196,
   'token': 8792,
   'token_str': '1851',
   'sequence': '[CLS] continent of america was discovered in 1851 by [MASK]. [SEP]'},
  {'score': 0.012034972198307514,
   'token': 8165,
   'token_str': '1859',
   'sequence': '[CLS] continent of america was discovered in 1859 by [MASK]. [SEP]'},
  {'score': 0.01202911976724863,
   'token': 6717,
   'token_str': '1864',
   'sequence': '[CLS] continent of america was discovered in 1864 by [MASK]. [SEP]'},
  {'score': 0.010807191021740437,
   'token': 6929,
   'token_str': '1886',
   'sequence': '[CLS] continent of america was discovered in 1886 by [MASK]. [SEP]'},
  {'score': 0.010435503907501698,
   'token': 10081,
   'token_str': '1836',
   'sequence': '[CLS] continent of america was discovered in 1836 by [MASK]. [SEP]'},
  {'score': 0.010013565421104431,
   'token': 8933,
   'token_str': '1853',
   'sequence': '[CLS] continent of america was discovered in 1853 by [MASK]. [SEP]'},
  {'score': 0.00962687563151

In [43]:
filed = unmasker("Continent of America was discovered by [MASK].", top_k=5)
print_unmasker_output(filed)

1. continent of america was discovered by europeans. 	  europeans, (0.05)
2. continent of america was discovered by c. 	  c, (0.05)
3. continent of america was discovered by fr. 	  fr, (0.04)
4. continent of america was discovered by columbus. 	  columbus, (0.03)
5. continent of america was discovered by ptolemy. 	  ptolemy, (0.03)


## 2. část - BERT contextualized word embeddings

BERT dokumentace obsahuje také návod jak použít tento model pro získání word embeddingů. Vyzkoušejte použití stejného slova v různém kontextu a podívejte se, jak se mění kosinova podobnost embeddingů v závislosti na kontextu daného slova.

Podívejte se na výstup tokenizeru před vstupem do BERT modelu - kolik tokenů bylo vytvořeno pro větu "Hello, this is Bert."? Zdůvodněte jejich počet.

<br>
Pozn.: Vyřešení předchozí otázky Vám pomůže zjistit, který vektor z výstupu pro cílové slovo použít.

In [46]:
nlp = pipeline('feature-extraction', model=model_name)
print(torch.Tensor(nlp("Hello")).shape)
print(torch.Tensor(nlp("Hello,")).shape)
print(torch.Tensor(nlp("Hello, this is Bert.")).shape)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([1, 3, 768])
torch.Size([1, 4, 768])
torch.Size([1, 8, 768])


### Výstup z tokenizéru:
- Embeding vektor má dimenzi 768.
- První embedding `[CLS]` je speciální embedding pro začátek věty.
- Poslední embedding `[SEP]` je speciální embedding pro konec věty.

Takže věta *"Hello, this is Bert."* obsahuje 4x slova, 2x interpunkci a 2x (start, stop) => 8 embedingů.

In [92]:
# Porovnání cosinusové vzdálenosti mezi vektory
def cos_sim(a, b):
    return torch.nn.functional.cosine_similarity(a, b, dim=0)

# Vytvoření vzorových vět
sentences = [
    ("desert, poušt", "Desert is a dry area with little water and few plants.", 1),
    ("wasteland, pustina", "The wasteland was covered in sand.", 2), # strange embeding lenght
    ("void, prázdnota", "This place is void of all life.", 4),
    ("desolated, zničena", "The local landscape was desolated.", 5), # strange embeding lenght
    ("desert, dezertovat", "The soldier deserted.", 3),
    ("defector, zběh", "This man is a defector.", 5),
    ("dessert, zákusek", "Dessert is a sweet food served at the end of a meal.", 1),
    ("cake, dort", "This is my birthday cake.", 5),
    ("dessert, zákusek", "I would like to order a dessert.", 7)
]

In [93]:
# Testovací výpis anotací a slov z indexů ve větě.
for anotation, sentence, word_index in sentences:
    print(anotation, ", picked word: ", sentence.split(" ")[word_index-1], 
        "\t Sentece lenght:", torch.Tensor(nlp(sentence)).shape[1])

desert, poušt , picked word:  Desert 	 Sentece lenght: 14
wasteland, pustina , picked word:  wasteland 	 Sentece lenght: 10
void, prázdnota , picked word:  void 	 Sentece lenght: 10
desolated, zničena , picked word:  desolated. 	 Sentece lenght: 10
desert, desertovat , picked word:  deserted. 	 Sentece lenght: 6
defector, zběh , picked word:  defector. 	 Sentece lenght: 9
dessert, zákusek , picked word:  Dessert 	 Sentece lenght: 15
cake, dort , picked word:  cake. 	 Sentece lenght: 8
dessert, zákusek , picked word:  dessert. 	 Sentece lenght: 10


In [94]:
# Vytvoření embeddingů pro všechny věty
embeddings = []
for anotation, sentence, word_index in sentences:
    embeddings.append(torch.Tensor(nlp(sentence))[0][word_index])

# Spočte se kosínová vzdálenost mezi kombinacemi vět
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            print(f"{sentences[i][0]} - {sentences[j][0]}: {cos_sim(embeddings[i], embeddings[j]):.2f}")


desert, poušt - wasteland, pustina: 0.20
desert, poušt - void, prázdnota: 0.13
desert, poušt - desolated, zničena: 0.06
desert, poušt - desert, desertovat: 0.15
desert, poušt - defector, zběh: 0.04
desert, poušt - dessert, zákusek: 0.69
desert, poušt - cake, dort: 0.01
desert, poušt - dessert, zákusek: 0.09
wasteland, pustina - desert, poušt: 0.20
wasteland, pustina - void, prázdnota: 0.43
wasteland, pustina - desolated, zničena: 0.54
wasteland, pustina - desert, desertovat: 0.31
wasteland, pustina - defector, zběh: 0.42
wasteland, pustina - dessert, zákusek: 0.06
wasteland, pustina - cake, dort: 0.28
wasteland, pustina - dessert, zákusek: 0.16
void, prázdnota - desert, poušt: 0.13
void, prázdnota - wasteland, pustina: 0.43
void, prázdnota - desolated, zničena: 0.60
void, prázdnota - desert, desertovat: 0.45
void, prázdnota - defector, zběh: 0.41
void, prázdnota - dessert, zákusek: 0.06
void, prázdnota - cake, dort: 0.32
void, prázdnota - dessert, zákusek: 0.29
desolated, zničena - des

## Bonus - Vizualizace slovních  embeddingů

Vizualizujte slovní embeddingy - mění se jejich pozice v závislosti na kontextu tak, jak byste očekávali? Pokuste se vizualizovat i některá slova, ke kterým by se podle vás cílové slovo mělo po změně kontextu přiblížit.

In [96]:
# TSNE
import numpy as np
tsne = TSNE(n_components=2, perplexity=2).fit_transform(torch.stack(embeddings))

plot = px.scatter(tsne[:, 0], tsne[:, 1], text=[s[0] for s in sentences])
plot.update_coloraxes(showscale=False)
plot.layout.template = 'plotly'
plot

/home/pytel/.local/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/home/pytel/.local/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.

